In [1]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [8]:
df=pd.read_csv('Data/redfin_2021-08-01-12-21-52_ForSale_and_Sold.csv')

In [11]:
df.head()

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL (SEE http://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING),SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
0,MLS Listing,NaN,Single Family Residential,2986 Bella Dr,Concord,CA,94519,729000,3,2.5,...,Active,August-1-2021 02:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/Concord/2986-Bella-Dr...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961231,N,Y,37.974726,-122.023066
1,MLS Listing,NaN,Single Family Residential,4747 Cunningham St,Oakland,CA,94619,849000,3,2.0,...,Active,August-1-2021 02:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/Oakland/4747-Cunningh...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961227,N,Y,37.786508,-122.182077
2,MLS Listing,NaN,Single Family Residential,14940 Riverdale St,San Leandro,CA,94578,799000,3,2.5,...,Active,August-1-2021 01:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/San-Leandro/14940-Riv...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40960879,N,Y,37.693961,-122.135551
3,MLS Listing,NaN,Single Family Residential,5366 Leona St,Oakland,CA,94619,995000,4,3.0,...,Active,August-1-2021 02:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/Oakland/5366-Leona-St...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961188,N,Y,37.786440,-122.175204
4,MLS Listing,NaN,Single Family Residential,1833 Leo Ln,Concord,CA,94521,949000,4,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Concord/1833-Leo-Ln-9...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961152,N,Y,37.959092,-121.952399


In [24]:
df_Oakland = df[df.CITY=='Oakland']

In [33]:
columns = list(df.columns)

In [50]:
def underscore (lst,rep="_"):
    for n in range(len(lst)):
        lst[n]=lst[n].replace(" ", rep)

In [51]:
underscore(columns)

In [55]:
columns[20]='URL'

In [56]:
columns

['SALE_TYPE',
 'SOLD_DATE',
 'PROPERTY_TYPE',
 'ADDRESS',
 'CITY',
 'STATE_OR_PROVINCE',
 'ZIP_OR_POSTAL_CODE',
 'PRICE',
 'BEDS',
 'BATHS',
 'LOCATION',
 'SQUARE_FEET',
 'LOT_SIZE',
 'YEAR_BUILT',
 'DAYS_ON_MARKET',
 '$/SQUARE_FEET',
 'HOA/MONTH',
 'STATUS',
 'NEXT_OPEN_HOUSE_START_TIME',
 'NEXT_OPEN_HOUSE_END_TIME',
 'URL',
 'SOURCE',
 'MLS#',
 'FAVORITE',
 'INTERESTED',
 'LATITUDE',
 'LONGITUDE']

In [57]:
df_Oakland.columns=columns

In [58]:
df_Oakland.head()

,SALE_TYPE,SOLD_DATE,PROPERTY_TYPE,ADDRESS,CITY,STATE_OR_PROVINCE,ZIP_OR_POSTAL_CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT_OPEN_HOUSE_START_TIME,NEXT_OPEN_HOUSE_END_TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,MLS Listing,NaN,Single Family Residential,4747 Cunningham St,Oakland,CA,94619,849000,3,2.0,...,Active,August-1-2021 02:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/Oakland/4747-Cunningh...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961227,N,Y,37.786508,-122.182077
3,MLS Listing,NaN,Single Family Residential,5366 Leona St,Oakland,CA,94619,995000,4,3.0,...,Active,August-1-2021 02:00 PM,August-1-2021 04:00 PM,http://www.redfin.com/CA/Oakland/5366-Leona-St...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40961188,N,Y,37.786440,-122.175204
8,MLS Listing,NaN,Single Family Residential,7955 Sanford St,Oakland,CA,94605,889000,3,2.0,...,Active,NaN,NaN,http://www.redfin.com/CA/Oakland/7955-Sanford-...,BAREIS,321060171,N,Y,37.771554,-122.154536
14,PAST SALE,July-14-2021,Single Family Residential,4798 Fair Ave,Oakland,CA,94619,976000,4,2.5,...,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/4798-Fair-Ave...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40954728,N,Y,37.785537,-122.183673
24,PAST SALE,May-7-2021,Single Family Residential,2210 Vicksburg Ave,Oakland,CA,94601,789000,3,2.0,...,Sold,NaN,NaN,http://www.redfin.com/CA/Oakland/2210-Vicksbur...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40943373,N,Y,37.776937,-122.203449


In [63]:
Oakland_ForSale = df_Oakland[df_Oakland.SOLD_DATE.isnull()]

In [66]:
df_Oakland.SOLD_DATE.isnull()

1       True
3       True
8       True
14     False
24     False
       ...  
337     True
341     True
344     True
346     True
348     True
Name: SOLD_DATE, Length: 70, dtype: bool

In [5]:
kaggle competitions download -c house-prices-advanced-regression-techniques

SyntaxError: invalid syntax (<ipython-input-5-1cca48ebbb57>, line 1)

In [4]:
import kaggle